In [ ]:
 # 

In [1]:
# fine annotate lineage 
import scanpy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import os,sys
import anndata
import warnings 
warnings.filterwarnings('ignore')



In [2]:
sc.logging.print_header()
import matplotlib.pyplot as plt
sc.settings.set_figure_params(dpi = 200, color_map = 'RdYlBu_r', dpi_save = 300, format = 'pdf',fontsize=5)
plt.rcParams["figure.figsize"] = [6,6]
# sns.set_palette('colorblind')

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.9.1 pandas==1.4.4 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.9.11 pynndescent==0.5.7


# integrate spatial object 

In [3]:
adata_paed = sc.read_h5ad('/nfs/team205/vk8/projects/thymus_atlas/results/thymus_atlas_v2_anno_hyper2_paed_CITE_v11_v2/cell2location_map/sp.h5ad')

In [4]:
# add 5% quantile, representing confident cell abundance, 'at least this amount is present', 
# to adata.obs with nice names for plotting
adata_paed.obs[adata_paed.uns['mod']['factor_names']] = adata_paed.obsm['q05_cell_abundance_w_sf']

In [5]:
# adata_paed.obs['tot_cell_abundance'] = adata_paed.uns["mod"]["post_sample_means"]["w_sf"].sum(1).flatten()
adata_paed.obs['detection_sensit']  = adata_paed.uns["mod"]["post_sample_q05"]["detection_y_s"]

In [6]:
# adata_paed_filt = adata_paed[adata_paed.obs['tot_cell_abundance']>30,:].copy()
adata_paed_filt = adata_paed[adata_paed.obs['n_genes_by_counts']>1000,:].copy()

adata_paed_filt

AnnData object with n_obs × n_vars = 29946 × 14786
    obs: 'Sample', 'Sample_hr', 'SampleID', 'SlideID', 'Position', 'in_tissue', 'array_row', 'array_col', 'x', 'y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'percent_mito', 'annotations_level_0', 'annotations_level_0_number', 'annotations_level_1', 'annotations_level_1_number', 'annotations_lobules_0', 'annotations_lobules_0_number', 'L2_dist_annotations_level_0_Artifacts', 'L2_dist_annotations_level_0_Cortex', 'L2_dist_annotations_level_0_Edge', 'L2_dist_annotations_level_0_Medulla', 'L2_dist_annotations_level_1_HS', 'L2_dist_annotations_level_1_PVS', 'L2_dist_annotations_level_1_fat', 'L2_dist_annotations_level_1_vessels', 'L2_dist_annotations_level_0_lymph', 'L2_dist_annotations_level_1_unassigned', 'cm', 'cc', 'cma_v2', 'manual_bin_cma_v2', 'manual_bin_cma_v2_int', 'Image_name', 'Image_jpg', 'section_thickness (um)', 'permebialisation(min)', 'RIN/DV200', 'Visium_type', 'Funding', 'Sequen

In [7]:

## remove meduala less lobules 
adata_paed_filt.obs['full_lobule'] = 0
for s in adata_paed_filt.obs['SampleID'].cat.categories.tolist():
    sample = adata_paed_filt[adata_paed_filt.obs['SampleID'].isin([s])]
    for l in sample.obs['annotations_lobules_0'].cat.categories:
        lobule = sample.obs['annotations_lobules_0'] == l
        med = np.where(sample.obs['annotations_level_0'][lobule] == 'Medulla')[0]
        cortex = np.where(sample.obs['annotations_level_0'][lobule] == 'Cortex')[0]
        if (cortex.size>=5) & (med.size>=5):
            adata_paed_filt.obs['full_lobule'][lobule.iloc[np.where(lobule)].index] = 1
adata_paed_filt.obs['full_lobule'] = adata_paed_filt.obs['full_lobule'].astype('category')
adata_paed_filt.obs['full_lobule'].value_counts()
adata_paed_filt = adata_paed_filt[adata_paed_filt.obs['full_lobule']==1].copy()

adata_paed_filt


AnnData object with n_obs × n_vars = 24779 × 14786
    obs: 'Sample', 'Sample_hr', 'SampleID', 'SlideID', 'Position', 'in_tissue', 'array_row', 'array_col', 'x', 'y', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'percent_mito', 'annotations_level_0', 'annotations_level_0_number', 'annotations_level_1', 'annotations_level_1_number', 'annotations_lobules_0', 'annotations_lobules_0_number', 'L2_dist_annotations_level_0_Artifacts', 'L2_dist_annotations_level_0_Cortex', 'L2_dist_annotations_level_0_Edge', 'L2_dist_annotations_level_0_Medulla', 'L2_dist_annotations_level_1_HS', 'L2_dist_annotations_level_1_PVS', 'L2_dist_annotations_level_1_fat', 'L2_dist_annotations_level_1_vessels', 'L2_dist_annotations_level_0_lymph', 'L2_dist_annotations_level_1_unassigned', 'cm', 'cc', 'cma_v2', 'manual_bin_cma_v2', 'manual_bin_cma_v2_int', 'Image_name', 'Image_jpg', 'section_thickness (um)', 'permebialisation(min)', 'RIN/DV200', 'Visium_type', 'Funding', 'Sequen

In [8]:
celltypes = adata_paed_filt.uns['mod']['factor_names']
cellab_paed = sc.AnnData(adata_paed_filt.obs[celltypes], 
                         obs = adata_paed_filt.obs.drop(celltypes, axis = 1),obsm = adata_paed_filt.obsm)
sc.pp.normalize_total(cellab_paed,target_sum=100)
cellab_paed.var_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '615', '616', '617', '618', '619', '620', '621', '622', '623', '624'],
      dtype='object', length=625)

In [35]:
# version 3 hybrid
import scipy 

cma_cluster = {}
for c in cellab_paed.var_names:
    # find the spots that have the highest expression of that cell
    quantile = np.quantile(cellab_paed.X[:,np.where(cellab_paed.var_names==c)],0.95)
    cells_above = np.where(cellab_paed.X[:,np.where(cellab_paed.var_names==c)]>quantile)[0]
    obs_ind = np.where(cellab_paed.obs.columns == 'cma_v2')[0]
    cell_abn = np.squeeze(cellab_paed.X[cells_above,np.where(cellab_paed.var_names==c)])                      
    # get the cma value for these 
    cma_value =  np.squeeze(np.array(cellab_paed.obs.iloc[cells_above,obs_ind]))
    cma_cluster[c] = np.dot(cell_abn,(np.exp2(cma_value-0.16)-1))/np.sum(cell_abn)



# space_map
# cma_cluster
# print(len(cells_above))
# cma_cluster

In [10]:
adata_paed_cells = sc.read_h5ad('/nfs/team205/vk8/projects/thymus_atlas/results/thymus_atlas_v2_anno_hyper2_paed_CITE_v11_v2/reference_signatures/sc.h5ad')
adata_paed_cells

AnnData object with n_obs × n_vars = 123873 × 14786
    obs: 'n_genes', 'sample_barcode', 'batch', 'chemistry', 'age_group', 'age_numeric', 'sampleID_TCRab', 'path_TCRab', 'type', 'study', 'sample', 'enrichment', 'doublet_score', 'predicted_doublet', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'pct_counts_hb', 'percent_mt2', 'n_counts', 'barcode_sequence', 'log1p_n_genes_by_counts', 'log1p_total_counts', 'log1p_total_counts_mt', 'status', 'assignment', 'log_prob_singleton', 'log_prob_doublet', 'louvian_1', 'louvian_05', 'latent_cell_probability', 'latent_RT_efficiency', 'log1p_n_counts', 'log1p_n_genes', 'percent_mito', 'n_counts_mito', 'percent_ribo', 'n_counts_ribo', 'percent_hb', 'n_counts_hb', 'percent_top50', 'n_counts_raw', 'log1p_n_counts_raw', 'n_genes_raw', 'log1p_n_genes_raw', 'percent_mito_raw', 'n_counts_mito_raw', 'percent_ribo_raw', 'n_counts_ribo_raw', 'percent_hb_raw', 'n_counts_hb_r

In [36]:
adata_paed_cells.obs['mean_hyper_cma_exp'] = adata_paed_cells.obs['hyper_leiden'].map(cma_cluster)
adata_paed_cells.obs['mean_hyper_cma_exp'] = adata_paed_cells.obs['mean_hyper_cma_exp'].astype('float32')


In [42]:
adata_paed_cells.obs.to_csv('/nfs/team205/ny1/ThymusSpatialAtlas/new_Figure6/cite_paed_hyper_mapping.csv')

In [493]:
# adata_paed_cells.obs.to_csv('HSTA_paed_v21_anno.csv')
adata_paed_cells.write_h5ad('/nfs/team205/ny1/ThymusSpatialAtlas/Figure5/Latest_version/HSTA_paed_v21_anno_hybrid.h5ad')